In [1]:
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report
import pandas as pd

# Filtrar dados e definir features e target
data = pd.read_excel('filtered_data_3.xlsx')
data.fillna(0, inplace=True)  # Substitui todos os NaNs por 0
data.head()

,Unnamed: 0,id_integer,role,years_exp,org_size,org_field,agile_methods_filtered,use_metrics_planning,use_metrics_review,use_metrics_weekly,...,metric_9,metric_10,metric_11,metric_13,metric_14,metric_15,metric_16,metric_17,metric_18,combined_text
0,0,1,Product manager,0 a 5,Microempresa,energia,"['Scrum', 'Kanban']",Sim,0,Sim,...,0,0,0,0,0,0,0,0,0,Product manager energia Scrum Kanban
1,1,2,Desenvolvedor(a),0 a 5,Microempresa,Financeiro,"['Scrum', 'Kanban']",Sim,Sim,0,...,0,0,0,0,0,0,0,0,0,Desenvolvedor(a) Financeiro Scrum Kanban
2,2,3,Product manager,0 a 5,Pequena empresa,Eventos,['Scrum'],0,Sim,0,...,0,0,0,0,0,0,0,0,0,Product manager Eventos Scrum
3,3,4,Team leader,6 a 9,Microempresa,gestão,"['Scrum', 'Kanban']",Sim,0,0,...,0,0,0,0,0,0,0,0,0,Team leader gestão Scrum Kanban
4,4,5,Product manager,0 a 5,Pequena empresa,Agronegocio,"['Scrum', 'Kanban']",Sim,Sim,0,...,0,0,0,0,0,0,0,0,0,Product manager Agronegocio Scrum Kanban


In [2]:

data_filtered = data.dropna(subset=['metrics_category'])

X = data_filtered[['role', 'years_exp', 'org_size', 'org_field', 'agile_methods_filtered', 
                   'use_metrics_planning', 'use_metrics_review', 'use_metrics_weekly', 
                   'use_metrics_daily', 'use_metrics_retro']].astype(str)
y = data_filtered['metrics_category']

X_combined = X.apply(lambda row: ' '.join(row.values), axis=1)
data['combined'] = X_combined
data.head()


,Unnamed: 0,id_integer,role,years_exp,org_size,org_field,agile_methods_filtered,use_metrics_planning,use_metrics_review,use_metrics_weekly,...,metric_10,metric_11,metric_13,metric_14,metric_15,metric_16,metric_17,metric_18,combined_text,combined
0,0,1,Product manager,0 a 5,Microempresa,energia,"['Scrum', 'Kanban']",Sim,0,Sim,...,0,0,0,0,0,0,0,0,Product manager energia Scrum Kanban,Product manager 0 a 5 Microempresa energia ['S...
1,1,2,Desenvolvedor(a),0 a 5,Microempresa,Financeiro,"['Scrum', 'Kanban']",Sim,Sim,0,...,0,0,0,0,0,0,0,0,Desenvolvedor(a) Financeiro Scrum Kanban,Desenvolvedor(a) 0 a 5 Microempresa Financeiro...
2,2,3,Product manager,0 a 5,Pequena empresa,Eventos,['Scrum'],0,Sim,0,...,0,0,0,0,0,0,0,0,Product manager Eventos Scrum,Product manager 0 a 5 Pequena empresa Eventos ...
3,3,4,Team leader,6 a 9,Microempresa,gestão,"['Scrum', 'Kanban']",Sim,0,0,...,0,0,0,0,0,0,0,0,Team leader gestão Scrum Kanban,Team leader 6 a 9 Microempresa gestão ['Scrum'...
4,4,5,Product manager,0 a 5,Pequena empresa,Agronegocio,"['Scrum', 'Kanban']",Sim,Sim,0,...,0,0,0,0,0,0,0,0,Product manager Agronegocio Scrum Kanban,Product manager 0 a 5 Pequena empresa Agronego...


In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [4]:
tfidf = TfidfVectorizer(stop_words='english')

# Ajustar e transformar o conjunto de dados X em uma matriz TF-IDF
tfidf_matrix = tfidf.fit_transform(X_combined)


print(tfidf_matrix)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 2246 stored elements and shape (210, 138)>
  Coords	Values
  (0, 108)	0.2769800094895152
  (0, 88)	0.22763816996634262
  (0, 93)	0.5470601244706502
  (0, 51)	0.6788570480977121
  (0, 115)	0.1417255920989659
  (0, 80)	0.1577449166023222
  (0, 122)	0.2576269865718727
  (1, 93)	0.6199340686168319
  (1, 115)	0.16060487505291599
  (1, 80)	0.1787581356757194
  (1, 122)	0.29194550804728503
  (1, 37)	0.644720230676435
  (1, 58)	0.2388241991396525
  (2, 108)	0.3021644281998328
  (2, 88)	0.24833617989654963
  (2, 115)	0.15461199736686357
  (2, 122)	0.28105173088053736
  (2, 104)	0.42094069386704547
  (2, 49)	0.13593329914841315
  (2, 56)	0.7405821530078199
  (3, 93)	0.5448076036532893
  (3, 115)	0.14114203677794038
  (3, 80)	0.15709540169055008
  (3, 122)	0.2565662070991801
  (3, 127)	0.267736590691703
  :	:
  (207, 23)	0.28569636836722995
  (207, 128)	0.6756633757807515
  (207, 116)	0.31833121755046423
  (208, 88)	0.24862862228970958

In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import classification_report

# Filtrar os dados e definir features e target
data_filtered = data.dropna(subset=['metrics_category'])
X = data_filtered[['role', 'years_exp', 'org_size', 'org_field', 'agile_methods_filtered', 
                   'use_metrics_planning', 'use_metrics_review', 'use_metrics_weekly', 
                   'use_metrics_daily', 'use_metrics_retro']].astype(str)

# Criar colunas binárias para cada categoria de métricas usando one-hot encoding
y = data_filtered['metrics_category'].str.get_dummies(sep=', ')

# Combinar todas as colunas de X em uma única coluna de texto
X_combined = X.apply(lambda row: ' '.join(row.values), axis=1)

# Inicializar o TfidfVectorizer
tfidf = TfidfVectorizer()

# Ajustar e transformar o conjunto de dados X em uma matriz TF-IDF
tfidf_matrix = tfidf.fit_transform(X_combined)

# Dividir os dados vetorizados em conjuntos de treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(tfidf_matrix, y, test_size=0.2, random_state=42)

# Criar o classificador RandomForest como um classificador multi-label
classifier = MultiOutputClassifier(RandomForestClassifier(n_estimators=100, random_state=42))

# Treinar o modelo
classifier.fit(X_train, y_train)

# Fazer previsões no conjunto de teste
y_pred = classifier.predict(X_test)

# Gerar e imprimir o relatório de classificação
report = classification_report(y_test, y_pred, target_names=y.columns)
print(report)


                             precision    recall  f1-score   support

                    Cliente       0.71      0.33      0.45        15
     Cronograma e progresso       0.75      0.90      0.82        30
Nenhuma das respostas acima       0.00      0.00      0.00         0
                    Pessoas       0.36      0.24      0.29        21
                   Processo       0.57      0.55      0.56        22
                    Produto       0.73      0.97      0.83        31
                 Tecnologia       0.65      0.50      0.56        22

                  micro avg       0.66      0.64      0.65       141
                  macro avg       0.54      0.50      0.50       141
               weighted avg       0.64      0.64      0.62       141
                samples avg       0.65      0.64      0.62       141



/Users/viniciuslunardifarias/projects/ufsc/tcc/tcc_env/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/viniciuslunardifarias/projects/ufsc/tcc/tcc_env/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/viniciuslunardifarias/projects/ufsc/tcc/tcc_env/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this b

In [6]:
# Definir um exemplo de usuário para ser classificado
user_data = {
    'role': 'Product manager',
    'years_exp': '0 a 5',
    'org_size': 'Microempresa',
    'org_field': 'energia',
    'agile_methods_filtered': "['Scrum', 'Kanban']",
    'use_metrics_planning': 'Sim',
    'use_metrics_review': 'Sim',
    'use_metrics_weekly': 'Não',
    'use_metrics_daily': 'Não',
    'use_metrics_retro': 'Sim'
}

# Combinar as características do usuário em uma única string
user_combined = ' '.join(user_data.values())

# Transformar o exemplo de usuário com o TfidfVectorizer ajustado anteriormente
user_tfidf = tfidf.transform([user_combined])

# Fazer a previsão usando o classificador treinado
user_prediction = classifier.predict(user_tfidf)

# Transformar a previsão em um formato legível (0 ou 1 para cada categoria)
user_prediction_labels = y.columns[(user_prediction == 1).flatten()]

# Exibir as categorias de métricas previstas para o usuário
print(user_prediction_labels)
print("Categorias de métricas recomendadas para o usuário:")
for label in user_prediction_labels:
    text_rec += label + ' '
    print(f"- {label}")


Index(['Cliente', 'Cronograma e progresso', 'Produto', 'Tecnologia'], dtype='object')
Categorias de métricas recomendadas para o usuário:


NameError: name 'text_rec' is not defined

In [26]:

# Carregar o dataset com as métricas
metrics_data = pd.read_excel('metrics.xlsx')

# Preencher valores nulos nas colunas 'Categories' e 'Description' com strings vazias
metrics_data['Categories'] = metrics_data['Categories'].fillna('')

# Combinar as categorias e descrições para criar uma base de conteúdo
metrics_data['content'] = metrics_data['Categories'] 

# Inicializar o TfidfVectorizer e ajustar aos dados de conteúdo das métricas
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix_metrics = tfidf_vectorizer.fit_transform(metrics_data['content'])

In [27]:
from sklearn.metrics.pairwise import cosine_similarity


In [30]:
def recommend_metrics(user_profile, top_n=5):
    # Transformar o perfil do usuário em um vetor TF-IDF
    user_tfidf = tfidf_vectorizer.transform([user_profile])
    
    # Calcular a similaridade do cosseno entre o perfil do usuário e as métricas
    cosine_similarities = cosine_similarity(user_tfidf, tfidf_matrix_metrics).flatten()
    
    # Obter os índices das métricas mais semelhantes
    similar_indices = cosine_similarities.argsort()[-top_n:][::-1]
    
    # Obter as métricas recomendadas e o grau de similaridade
    recommended_metrics = metrics_data.iloc[similar_indices].copy()
    recommended_metrics['similarity (%)'] = (cosine_similarities[similar_indices] * 100).round(2)
    
    return recommended_metrics

In [32]:
# Simular um perfil de usuário (exemplo baseado nas categorias de métricas)
user_profile = "Cronograma e progresso, Tecnologia, Produto"

# Obter recomendações para o perfil de usuário
recommended_metrics = recommend_metrics(user_profile)

# Mostrar as métricas recomendadas junto com a similaridade
print("\nMétricas recomendadas para o perfil do usuário:")
print(recommended_metrics[['metric', 'Categories', 'Description', 'similarity (%)']])


Métricas recomendadas para o perfil do usuário:
                       metric                                   Categories  \
4                  Throughput  Cronograma e progresso, Produto, Tecnologia   
5   Number of remaining tasks  Cronograma e progresso, Produto, Tecnologia   
17       Technical Efficiency                          Produto, Tecnologia   
2                    Burndown           Cronograma e progresso, Tecnologia   
7                    Velocity  Cronograma e progresso, Produto, Processo,    

                                          Description  similarity (%)  
4   Density of bugs that were corrected within the...          100.00  
5   The “Project Size Remaining” metric closely re...          100.00  
17        "Technical efficiency (output/input)\nAH/C"           82.88  
2                                                 NaN           81.03  
7   "Velocity measures work completed in a\nSprint...           69.24  


In [40]:
# Definir um exemplo de usuário para ser classificado
user_data = {
    'role': 'Product manager',
    'years_exp': '0 a 5',
    'org_size': 'Microempresa',
    'org_field': 'Telecom',
    'agile_methods_filtered': "['Scrum', 'Kanban']",
    'use_metrics_planning': 'Sim',
    'use_metrics_review': 'Sim',
    'use_metrics_weekly': 'Não',
    'use_metrics_daily': 'Não',
    'use_metrics_retro': 'Sim'
}

# Combinar as características do usuário em uma única string
user_combined = ' '.join(user_data.values())

# Transformar o exemplo de usuário com o TfidfVectorizer ajustado anteriormente
user_tfidf = tfidf.transform([user_combined])

# Fazer a previsão usando o classificador treinado
user_prediction = classifier.predict(user_tfidf)

# Converter a previsão em categorias
predicted_categories = y.columns[(user_prediction == 1).flatten()]
user_profile = ', '.join(predicted_categories)

# Exibir as categorias previstas
print("Categorias de métricas classificadas para o usuário:")
print(user_profile)

# Parte 4: Recomendação com Base nas Categorias Classificadas

# Obter recomendações para o perfil de usuário
recommended_metrics = recommend_metrics(user_profile)

# Mostrar as métricas recomendadas junto com a similaridade
print("\nMétricas recomendadas para o perfil do usuário:")
print(recommended_metrics[['metric', 'Categories', 'Description', 'similarity (%)']])

Categorias de métricas classificadas para o usuário:
Cliente, Cronograma e progresso, Produto

Métricas recomendadas para o perfil do usuário:
                        metric                                   Categories  \
25              Quality rating                             Produto, Cliente   
18              Valor agregado                             Produto, Cliente   
3             Outstanding bugs                             Produto, Cliente   
21  Number of Active Customers              Cronograma e progresso, Cliente   
7                     Velocity  Cronograma e progresso, Produto, Processo,    

                                          Description  similarity (%)  
25  The assessed quality rating\n"Quality of produ...           83.54  
18                                                NaN           83.54  
3   The purpose of this measure is to show the num...           83.54  
21  "Number of customers is a core measure\nfor cu...           81.76  
7   "Velocity measures